In [1]:
import time
import tempfile

from functools import reduce
import numpy as np
import scipy

from pyscf import gto, scf
from pyscf import lib, lo
from pyscf.lib import logger
from pyscf.data import elements
from pyscf.grad import rhf

In [2]:
amu2au = 1.6605402e-24/9.1093837015e-28
au2eV = 27.211386246

m1 = 23.0 * amu2au
m2 = 35.5 * amu2au

v1 =  -0.00055545
v2 = 0.00036022

k1 = 1/2 * m1 * v1**2
k2 = 1/2 * m2 * v2**2

print((k1+k2)*au2eV)

mol =   gto.Mole( atom='''
    Na    0.0000000    0.0000000    1.205
    Cl    0.0000000    0.0000000   -1.205
    '''
    , basis='3-21g', symmetry=False, cart=False).build()

mf = scf.RHF(mol)
mf.verbose = 0
mf.max_cycle = 100
mf.kernel()
dm = mf.make_rdm1()

0.2902414682120198


In [10]:
mass = np.array(((m1,m2)))
eV2au = 0.036749405469679
energy = 1.16
energy = energy * eV2au

# Convert vibration energy (eV) to momenta (Only for 2-atom molecule)
# Initially momenta is conserved: P_a = -P_b

def cal_velocity(momenta, mass):
    '''Calculate velocity from momenta'''
    mass_reci = np.reciprocal(mass).reshape((1,2))
    vel = momenta * mass_reci.T

    return vel

mass_r = (mass[0] + mass[1]) / (mass[0] * mass[1])
vector = mol.atom_coords()[0] - mol.atom_coords()[1]
direction = vector / (np.sqrt(vector[0]**2 + vector[1]**2 + vector[2]**2))
momenta_sq = energy / mass_r * 2 
momenta = np.sqrt(momenta_sq)
momenta_init = -momenta * np.array((direction,-direction))
vel = cal_velocity(momenta_init,mass)

print(vel)

[[-0.         -0.         -0.00111086]
 [ 0.          0.          0.00071972]]


In [12]:
(0.5*mass[0]*vel[0][2]**2+0.5*mass[1]*vel[1][2]**2)/eV2au

1.16

In [16]:
ene = np.linalg.norm(momenta_init[0])**2 * mass_r / 2
print(ene/eV2au)

1.1600000000000001
